In [1]:
import requests
def NewFromBBC():
    query_params = {
        "source": "bbc-news",
        "sortBy": "top",
        "apiKey": "0a584dc3ec024dec9d8bee6f741c79d2"
    }
    main_url = "http://newsapi.org/v2/top-headlines?country=in&category=business"
    res = requests.get(main_url,params = query_params)
    open_bbc_page = res.json()
    articles = open_bbc_page["articles"]
    results = []
    dis=[]
    for ar in articles:
        results.append(ar["title"])
        dis.append(ar["description"])
    return (results,dis)
    #for i in range(len(results)):
        #print(i+1,results[i])
    from win32com.client import Dispatch
    speak = Dispatch("SAPI.spvoice")
    speak.speak(results)
if __name__ == '__main__':
    res,disc=NewFromBBC()

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from dash.dependencies import Input, Output, State
import numpy as np
import yfinance
import re


external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Stock Price Analysis"

server = app.server
scaler=MinMaxScaler(feature_range=(0,1))
df_nse = pd.read_csv("./NSE-Tata-Global-Beverages-Limited.csv")
df_nse["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_nse.index=df_nse['Date']
data=df_nse.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])
for i in range(0,len(data)):
    new_data["Date"][i]=data['Date'][i]
    new_data["Close"][i]=data["Close"][i]
new_data.index=new_data.Date
new_data.drop("Date",axis=1,inplace=True)
dataset=new_data.values
train=dataset[0:987,:]
valid=dataset[987:,:]
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
x_train,y_train=[],[]
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
    
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

model=load_model("saved_model.h5")
inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)
X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)
train=new_data[:987]
valid=new_data[987:]
valid['Predictions']=closing_price


C:\Users\saura\AppData\Local\Temp/ipykernel_1852/3444053496.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\saura\AppData\Local\Temp/ipykernel_1852/3444053496.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\saura\AppData\Local\Temp/ipykernel_1852/3444053496.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions']=closing_price


In [3]:
import pandas as pd
from datetime import datetime
import plotly.graph_objs as go
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


##### Layouts for Tab 1 #####
# Dropdown list for single stock choice
def get_tab1_info_box():
	return html.Div([
				html.Div([
					html.Div(id='tab1-stock-price',
						     style={'width':'30%','display':'inline-block',
						            'font-size':'200%'}),
					html.Div(id='tab1-stock-price-change'),
					html.Div(id='tab1-stock-price-percentchange')
					],style={'width':'30%','display': 'inline-block',
			                 'vertical-align':'top'}),
				html.Div(style={'width':'15%','display': 'inline-block'}),
				html.Div(dcc.Dropdown(id='tab1-market-dropdown',
					                  options=tab1_markets,
					                  value=tab1_markets[0]['value'],
				                      style={'text-align':'left'}),
					     style={'width':'20%','display': 'inline-block',
			            		'vertical-align':'top'}),
				html.Div(style={'width':'5%','display': 'inline-block'}),
				html.Div([
					html.Div([
						html.Div(dcc.Input(id='tab1-ticker-input',value='',
						                   type='text'),
						                   style={'display': 'inline-block'}),
						html.Div(html.Button('Submit',id='tab1-submit'),
							     style={'display': 'inline-block'})
						]),
					html.Div(id='tab1-error-message', style={'color':'red'}, children='Error Box')
					],style={'width':'30%','display': 'inline-block'})
			])

##### Layout for Tab 2 #####
# Dropdown list for index and stock growth difference
def get_tab2_info_box():
	return html.Div([
			html.Div([dcc.Dropdown(id='tab2-index-choice',
				                   options=index_choice,
				                   value=index_choice[0]['value'],
								   style={'text-align':'left'})],
				style={'width':'30%','display': 'inline-block'}),
			html.Div(style={'width':'20%','display': 'inline-block'}),
			html.Div([dcc.Dropdown(id='tab2-stock-include',
				                   options=[],
				                   value=[],
				                   multi=True,
				                   style={'text-align':'left'})],
				style={'width':'50%','display': 'inline-block'})
		])

##### Layout for both Tab 1 and Tab 2 #####
# Layout for user to select time interval
def get_interval_layout(tab):
	id_name = tab+'-time-interval'
	layout = html.Div(
		dcc.RadioItems(id=id_name,
                       options=[{'label':'1 Month', 'value':'1mo'},
                                {'label':'3 Months', 'value':'3mo'},
                                {'label':'6 Months', 'value':'6mo'},
                                {'label':'YTD', 'value':'ytd'},
                                {'label':'1 Year', 'value':'1y'},
                                {'label':'3 Years', 'value':'3y'},
                                {'label':'5 Years', 'value':'5y'}
                       ],
                       value='ytd'
		), style={'text-align':'center'})
	return layout

# Layout for graph, include time interval and graph
def get_graph_layout(tab):
	layout = html.Div([
		html.Br(),
		get_interval_layout(tab),
		dcc.Graph(id=tab+'-vis')
		])
	return layout

# Layout for table for stats
def get_table_layout(tab):
	id_name = tab+'-table'
	return html.Div([html.Br(),html.Br(),html.Table(id=id_name)])

# Layout for stats and graph
def get_stats_graph_layout(tab):
	return html.Div([
		html.Div(get_table_layout(tab),
			     style={'width':'30%','display': 'inline-block',
			            'vertical-align':'top'}),
		html.Div(get_graph_layout(tab),
			     style={'width':'70%','display': 'inline-block',
			            'vertical-align':'top'})
		])

##### Generate Plots and Tables#####
# Generate Line charts for stocks and index
def getLinePlot(df, tab):
	traces = []
	for col in df.columns:
		if col != 'Date':
			traces.append({'x':df['Date'], 'y':df[col],'name':col,
				           'mode':'lines'})
	layout = {'xaxis':{'title':'Date'},
	          'hovermode':False}
	if tab == 1:
		layout['yaxis'] = {'title':'Price ($)'}

	elif tab == 2:
		layout['yaxis'] = {'title':'% Change', 'tickformat':'.0%'}
	return {'data':traces, 'layout':layout}

# Generate candlestick
def getCandlestick(df):
	data = []
	data.append(go.Candlestick(x=df['Date'], open=df['Open'],
		                       high=df['High'], low=df['Low'],
		                       close=df['Close']))
	layout = {'xaxis':{'title':'Date','rangeslider':{'visible':False}},
			  'yaxis':{'title':'Price ($)'},
	          'hovermode':False}
	return {'data':data, 'layout':layout}

# Generate Table for Tab 1 - Stock Stats
def getTab1Table(df, stock_info):
	last_day = df.iloc[-1,1:6]
	# Format the day range of price
	low_day = last_day['Low']
	high_day = last_day['High']
	range_day = f'{low_day:,.2f}'+' - '+f'{high_day:,.2f}'
	# Obtain and format 52 weeks range of price
	low_52weeks = df['Low'].min()
	high_52weeks = df['High'].max()
	range_52weeks = f'{low_day:,.2f}'+' - '+f'{high_day:,.2f}'

	# Format volume and average volume
	vol = last_day['Volume']
	vol = f'{vol:,.0f}'
	avg_vol = stock_info['averageVolume10days']
	avg_vol = f'{avg_vol:,.0f}'

	#Obtain shares outstanding
	shareOutstanding = stock_info['sharesOutstanding']

	# Calculate market cap and format it
	mktcap = last_day['Close']*shareOutstanding
	mktcap = f'{mktcap:,.0f}'

	# Format beta
	beta = stock_info['beta']
	beta = f'{beta:.2f}'

	# Format PE and Forward PE, if no PE, PE not in the dictionary
	if 'trailingPE' in stock_info:
		pe = stock_info['trailingPE']
		pe = f'{pe:.2f}'
	else:
		pe = 'N/A'
	if 'forwardPE' in stock_info:
		fpe = stock_info['forwardPE']
		fpe = f'{fpe:.2f}'
	else:
		fpe = 'N/A'

	# Format EPS
	eps = stock_info['trailingEps']
	eps = f'{eps:.2f}'

	# Format Profit margin
	margin = stock_info['profitMargins']
	margin = f'{margin:.2f}'

	# Prepare data for dividend rate
	if stock_info['dividendRate'] is None or stock_info['dividendRate']=='':
		dividend = 'N/A'
	else:
		dividend = stock_info['dividendRate']

	# Prepare data for ex-dividend date
	if stock_info['exDividendDate'] is not None:
		ex_dividend_date = datetime.fromtimestamp(stock_info['exDividendDate'])
		ex_dividend_date = ex_dividend_date.strftime('%m-%d-%Y')
	else: 
		ex_dividend_date = 'N/A'



	return html.Table([
		html.Tr([html.Td('Industry'), html.Td(),
		         html.Td(stock_info['industry'])]),
		html.Tr([html.Td('Previous Close'), html.Td(),
			     html.Td(stock_info['previousClose'])]),
		html.Tr([html.Td('Open'), html.Td(),
			     html.Td(last_day['Open'])]),
		html.Tr([html.Td('Day Range'), html.Td(),
			     html.Td(range_day)]),
		html.Tr([html.Td('52 Weeks Range'), html.Td(),
			     html.Td(range_52weeks)]),
		html.Tr([html.Td('Volume'), html.Td(),
			     html.Td(vol)]),
		html.Tr([html.Td('Average Volume'), html.Td(),
			     html.Td(avg_vol)]),
		html.Tr([html.Td('Market Capitalization'), html.Td(),
			     html.Td(mktcap)]),
		html.Tr([html.Td('Beta'), html.Td(),
			     html.Td(beta)]),
		html.Tr([html.Td('PE'), html.Td(),
			     html.Td(pe)]),
		html.Tr([html.Td('Forward PE'), html.Td(),
			     html.Td(fpe)]),
		html.Tr([html.Td('Earning Per Share (EPS)'), html.Td(),
			     html.Td(eps)]),
		html.Tr([html.Td('Profit Margin'), html.Td(),
			     html.Td(margin)]),
		html.Tr([html.Td('Dividend'), html.Td(),
			     html.Td(dividend)]),
		html.Tr([html.Td('Ex-Dividend Date'), html.Td(),
			     html.Td(ex_dividend_date)]),
		html.Tr([html.Td('Earning Per Share (EPS)'), html.Td(),
			     html.Td(f'{shareOutstanding:,.0f}')])
		])

# Generate Table for Tab 2 - Index
def getTab2Table(name, last_close, range_period, range_52weeks):
	if last_close >= 0:
		color = 'green'
	else:
		color = 'red'
	last_close = last_close = f'{last_close:.2f}%'
	return html.Table([
		html.Tr(html.Td(html.B(name))),
		html.Tr([html.Td('Close'),
			    html.Td(last_close, style={'color':color})]),
		html.Tr([html.Td('Period Range'),
			     html.Td(f'{range_period[0]:,.2f}'+ \
			     	     ' - '+f'{range_period[1]:,.2f}')]),
		html.Tr([html.Td('52 Weeks Range'),
			     html.Td(f'{range_52weeks[0]:,.2f}'+\
			     	     ' - '+f'{range_52weeks[1]:,.2f}')])
		])

##### Global Variables #####
# Dictionary for the dropdown list to select stock market
tab1_markets = [{'label':'Hong Kong', 'value':'hk'},
                {'label':'United States', 'value':'us'},
               {'label':'India', 'value':'ind'}]
# Dictionary for the dropdown list to select index in Tab 2
index_choice = [{'label':' Hong Kong: Heng Seng Index', 'value':'hsi'},
                {'label':' United States: S&P 500', 'value':'sp500'}]

In [ ]:
app.layout = html.Div(
    #style={
    #'background-image': 'url("/assets/d3.jpg")',
    #'background-repeat': 'no-repeat',
    #'background-position': 'center',
    #'background-size': '1920px 1080px'
    #},
        children=[
        html.Div(
            children=[
                html.P(children="📈📉📊", className="header-emoji"),
                html.H1(
                    children="Stock Price Analysis Dashboard", className="header-title"
                ),
                html.P(
                    children="Analying stock market and predictions.Copyright 2021",
                    className="header-description",
                ),
            ],
            className="header",
        ),
   
    dcc.Tabs(id="tabs", children=[
               
        dcc.Tab(label='NSE-TATAGLOBAL Stock Data',children=[
            html.Div([
                html.H2("Actual closing price",style={"textAlign": "center"}),
                dcc.Graph(
                    id="Actual Data",
                    figure={
                        "data":[
                            go.Scatter(
                                x=valid.index,
                                y=valid["Close"],
                                mode='lines'
                            )
                        ],
                        "layout":go.Layout(
                            title='lines plot',
                            xaxis={'title':'Date'},
                            yaxis={'title':'Closing Rate'}
                        )
                    }
                ),
                html.H2("LSTM Predicted closing price",style={"textAlign": "center"}),
                dcc.Graph(
                    id="Predicted Data",
                    figure={
                        "data":[
                            go.Scatter(
                                x=valid.index,
                                y=valid["Predictions"],
                                mode='lines'
                            )
                        ],
                        "layout":go.Layout(
                            title='lines plot',
                            xaxis={'title':'Date'},
                            yaxis={'title':'Closing Rate'}
                        )
                    }
                )                
            ])                
        ]),
        dcc.Tab(label='Stock Price', value='price-tab',children=[
			html.Div([html.H2(id='tab1-stock-name',
				           style={'width':'30%','display':'inline-block'}), 
				      html.H4(id='tab1-ticker', 
				           style={'width':'10%','display':'inline-block'})],
				      style={'width':'90%','margin':'auto'}),
			# Position 0, Title
			html.Div(get_tab1_info_box(),
				     style={'width':'90%','margin':'auto'}),
			# Position 1, Info and dropdown
			html.Div(get_stats_graph_layout('tab1'),
				     style={'width':'90%','margin':'auto'})
			# Position 2, Table of stats and graph
			]),
             dcc.Tab(label='NEWS',children=[
            html.Div([
                html.H2('•'+res[0],style={"textAlign": "left"}),
                html.H3("-"+disc[0],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[1],style={"textAlign": "left"}),
                html.H3('-'+disc[1],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[2],style={"textAlign": "left"}),
                html.H3('-'+disc[2],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[3],style={"textAlign": "left"}),
                html.H3('-'+disc[3],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[4],style={"textAlign": "left"}),
                html.H3('-'+disc[4],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[5],style={"textAlign": "left"}),
                html.H3('-'+disc[5],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[6],style={"textAlign": "left"}),
                html.H3('-'+disc[6],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2(res[7],style={"textAlign": "left"}),
                html.H3(disc[7],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[8],style={"textAlign": "left"}),
                html.H3('-'+disc[8],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
                html.H2('•'+res[9],style={"textAlign": "left"}),
                html.H3('-'+disc[9],style={"textAlign": "left"}),
                html.H1("__________________________________________________________________________________________________________________________________________________________________________________________________________"),
             
                        ],
                 )
    ])
])
])
def verify_ticker(ticker, mkt):
	# Identify which ticker is that
	# For Hong Kong Stocks
	if mkt == 'hk':
		tick = re.findall('^\d{1,5}$', ticker)
		if len(tick)>0:
			tick = str(tick[0])[::-1]
			while(len(tick)<4):
				tick += '0'
			tick = tick[::-1]
			tick += '.HK'
			return True, tick
		# If entered ticker invalid
		else:
			return False, None
	#
	elif mkt == 'us':
		tick = re.findall('^[A-Za-z]{1,4}$', ticker)
		if len(tick)>0:
			return True, tick[0].upper()
	elif mkt == 'ind':
		tick = re.findall('^[A-Za-z]{1,16}$', ticker)
		if len(tick)>0:
			return True, tick[0].upper()+'.NS'
		else:
			return False, None
	return False, None

# Obtain 50- ,100- and 200-day moving average
def getMA(stock, time, date_list):
	if 'mo' in time or time=='ytd' or time=='1y':
		df = stock.history(period='2y')
	elif time=='2y' or time=='3y' or time=='4y':
		df = stock.history(period='5y')
	else:
		df = stock.history(period='10y')
	df = df.reset_index()[['Date','Open','Low','High','Close']]
	df['MA50'] = df.Close.rolling(50).mean()
	df['MA100'] = df.Close.rolling(100).mean()
	df['MA200'] = df.Close.rolling(200).mean()

	df = df.loc[(df['Date']>=date_list.min()) & (df['Date']<=date_list.max())]
	return df

# Generate stock price and graph on Tab 1
@app.callback([Output('tab1-stock-name','children'), # Stock Name
	           Output('tab1-ticker','children'), # Ticker
	           Output('tab1-stock-price','children'), # Current Stock Price
	           Output('tab1-stock-price-change','children'), # Price Change
	           Output('tab1-stock-price-change','style'), 
	           # Price Change font colour
	           Output('tab1-stock-price-percentchange','children'), 
	           # Price Percent Change
	           Output('tab1-stock-price-percentchange','style'), 
	           # Price Precent Change font colour
	           Output('tab1-error-message','children'), # Error Message
	           Output('tab1-vis','figure'), # Stock Price Chart
	           Output('tab1-table','children')], # Table of Stock Stats
	          [Input('tab1-submit','n_clicks'), # Button
	           Input('tab1-time-interval','value')], # Time interval
	          [State('tab1-ticker-input','value'), # Ticker textbox input
	           State('tab1-market-dropdown', 'value')]) # Market Dropdown input
def get_ticker(n_clicks, time, ticker, mkt):
	# For default setting
	if ticker == '':
		return 'Please Enter a Stock Ticker', \
		       '','','',{'width':'20%', 'display':'inline-block'},'', \
		       {'width':'20%', 'display':'inline-block'},'', \
		       {'data':None}, None
	# Verify ticker format in respective to stock market
	stockFormat, ticker = verify_ticker(ticker, mkt)
	# Catch incorrect 
	if stockFormat is False:
		return 'Wrong Ticker', '#######', '$##.##', '##.##', \
		       {'width':'20%', 'display':'inline-block'}, '##.##%', \
		       {'width':'20%', 'display':'inline-block'}, \
		       'Error! Please try again.', {'data':None}, None
	# Obtain stock price and stats
	stock = yfinance.Ticker(ticker)
	# Catch if stock exists
	if stock.history(period='ytd').shape[0] == 0:
		return 'Wrong Ticker', '#######', '$##.##', '##.##', \
		       {'width':'20%', 'display':'inline-block'}, '##.##%', \
		       {'width':'20%', 'display':'inline-block'}, \
		       'Error! Please try again.', {'data':None}, None
	### Stock Stats for Info Box ###
	try: 
		# Name and price
		stock_name = stock.info['longName']
		price_list = stock.history(period=time)['Close'].tolist()
		price = f'{price_list[-1]:.2f}'
		# Price Change
		price_change = price_list[-1] - price_list[-2]
		price_percent_change = (price_list[-1]/price_list[-2])-1
		if price_change > 0:
			price_change_colour = {'color':'green'}
		else:
			price_change_colour = {'color':'red'}
		price_change_colour['display']= 'inline-block'
		price_change_colour['width']= '20%'
		price_change_colour['font-size'] = '150%'
		price_change = f'{price_change:.2f}'
		price_percent_change = f'{price_percent_change*100:,.2f}%'

		df = getMA(stock, time, 
			       stock.history(period=time).reset_index()['Date'])

		fig = getCandlestick(df)
		table = getTab1Table(stock.history(period=time).reset_index(),
			                 stock.info)

	except:
		return 'Sorry! Company Not Available', '#######', '$##.##', '##.##', \
		       {'width':'20%', 'display':'inline-block'}, '##.##%', \
		       {'width':'20%', 'display':'inline-block'}, \
		       'Error! Please try again another Company.', {'data':None}, None


	return stock_name, ticker, price, price_change, price_change_colour, \
	       price_percent_change, price_change_colour, '', fig, table
if __name__=='__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Oct/2021 13:34:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:10] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:10] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:10] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:19] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2021 13:34:45] "POST /_dash-update-component HTTP/1.1" 200 -
